In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import random

class GymRecommendationModel:
    def __init__(self):
        self.label_encoders = {}
        self.scaler = StandardScaler()
        self.data = None
        self.user_features_cols = ['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI', 'Level', 'Fitness Goal', 'Fitness Type']
        self.output_cols = ['Exercises', 'Diet', 'Equipment']

    def fit(self, data):
        """Preprocess the data (encode categorical features and scale numerical features)."""
        # Drop unnecessary columns
        self.data = data.drop(columns=['ID'], errors='ignore')

        # Encode categorical features
        categorical_cols = ['Sex', 'Hypertension', 'Diabetes', 'Level', 'Fitness Goal', 'Fitness Type']
        for col in categorical_cols:
            self.label_encoders[col] = LabelEncoder()
            self.data[col] = self.label_encoders[col].fit_transform(self.data[col])

        # Scale numerical features
        numerical_cols = ['Age', 'Height', 'Weight', 'BMI']
        self.data[numerical_cols] = self.scaler.fit_transform(self.data[numerical_cols])

    def _preprocess_user_input(self, user_input):
        """Preprocess user input to match the model's data format."""
        user_df = pd.DataFrame([user_input], columns=self.user_features_cols)
        
        # Scale numerical features
        numerical_cols = ['Age', 'Height', 'Weight', 'BMI']
        user_df[numerical_cols] = self.scaler.transform(user_df[numerical_cols])
        
        return user_df

    def predict(self, top_n=3):
        """Generate personalized workout and diet recommendations based on user input."""
        print("Please enter your details for a personalized workout and diet recommendation.")
        user_input = {
            'Sex': int(input("Enter Sex (Male: 1/Female: 0): ")),
            'Age': float(input("Enter Age: ")),
            'Height': float(input("Enter Height in meters (e.g., 1.75): ")),
            'Weight': float(input("Enter Weight in kg: ")),
            'Hypertension': int(input("Do you have Hypertension (Yes: 1/No: 0): ")),
            'Diabetes': int(input("Do you have Diabetes (Yes: 1/No: 0): ")),
            'BMI': float(input("Enter BMI: ")),
            'Level': int(input("Enter Level (Underweight: 3, Normal: 0, Overweight: 2, Obese: 1): ")),
            'Fitness Goal': int(input("Enter Fitness Goal (Weight Gain: 0, Weight Loss: 1): ")),
            'Fitness Type': int(input("Enter Fitness Type (Muscular Fitness: 1, Cardio Fitness: 0): "))
        }

        # Preprocess user input
        user_df = self._preprocess_user_input(user_input)

        # Calculate similarity scores
        user_features = self.data[self.user_features_cols]
        similarity_scores = cosine_similarity(user_features, user_df).flatten()

        # Retrieve top similar users and get the first recommendation
        similar_user_indices = similarity_scores.argsort()[-5:][::-1]
        similar_users = self.data.iloc[similar_user_indices]
        recommendation_1 = similar_users[self.output_cols].mode().iloc[0]

        # Simulate two additional recommendations by modifying input values
        simulated_recommendations = []
        for _ in range(2):
            modified_input = user_input.copy()
            modified_input['Age'] += random.randint(-5, 5)
            modified_input['Weight'] += random.uniform(-5, 5)
            modified_input['BMI'] += random.uniform(-1, 1)

            # Preprocess modified input
            modified_user_df = self._preprocess_user_input(modified_input)

            # Calculate similarity scores for modified input
            modified_similarity_scores = cosine_similarity(user_features, modified_user_df).flatten()
            modified_similar_user_indices = modified_similarity_scores.argsort()[-5:][::-1]
            modified_similar_users = self.data.iloc[modified_similar_user_indices]
            recommendation = modified_similar_users[self.output_cols].mode().iloc[0]

            # Ensure unique recommendations
            if not any(
                rec['Exercises'] == recommendation['Exercises'] and
                rec['Diet'] == recommendation['Diet'] and
                rec['Equipment'] == recommendation['Equipment']
                for rec in simulated_recommendations
            ):
                simulated_recommendations.append(recommendation)

        # Display recommendations
        print("\nRecommended Workout and Diet Plans based on your input:")
        print("\nRecommendation 1 (Exact match):")
        print("EXERCISES:", recommendation_1['Exercises'])
        print("EQUIPMENTS:", recommendation_1['Equipment'])
        print("DIET:", recommendation_1['Diet'])

        for idx, rec in enumerate(simulated_recommendations, start=2):
            print(f"\nRecommendation {idx} (Slight variation):")
            print("EXERCISES:", rec['Exercises'])
            print("EQUIPMENTS:", rec['Equipment'])
            print("DIET:", rec['Diet'])

        # Collect feedback for MRR
        feedback_matrix = []
        for i in range(len(simulated_recommendations) + 1):
            feedback = int(input(f"Was recommendation {i+1} relevant? (Yes: 1, No: 0): "))
            feedback_matrix.append(feedback)

        # Calculate MRR
        relevant_indices = [i + 1 for i, feedback in enumerate(feedback_matrix) if feedback == 1]
        mrr = np.mean([1 / rank for rank in relevant_indices]) if relevant_indices else 0.0

        print(f"\nMean Reciprocal Rank (MRR): {mrr:.2f}")

        return [recommendation_1] + simulated_recommendations

# Load data and instantiate the model
data = pd.read_csv("/Users/keerthanagc/Downloads/AI/gym recommendation.csv")
model = GymRecommendationModel()

# Fit the model
model.fit(data)

# Get predictions
recommendations = model.predict(top_n=3)

Please enter your details for a personalized workout and diet recommendation.

Recommended Workout and Diet Plans based on your input:

Recommendation 1 (Exact match):
EXERCISES: Squats, yoga, deadlifts, bench presses, and overhead presses
EQUIPMENTS: Light athletic shoes, resistance bands, light dumbbells and a Blood glucose monitor.
DIET: Vegetables: (Garlic, Roma Tomatoes, Capers, Green Papper, and Iceberg Lettuce); Protein Intake: (Cheese Sandwich, Baru Nuts, Beech Nuts, Squash Seeds, Mixed Teff, peanut butter, and jelly sandwich); Juice: (Apple juice, beetroot juice, and mango juice)

Recommendation 2 (Slight variation):
EXERCISES: Squats, deadlifts, bench presses, and overhead presses
EQUIPMENTS: Dumbbells and barbells
DIET: Vegetables: (Mixed greens, cherry tomatoes, cucumbers, bell peppers, carrots, celery, bell peppers);Protein Intake: (Chicken, fish, tofu, or legumes); Juice : (Green juice,kale, spinach, cucumber, celery, and apple)

Recommendation 3 (Slight variation):
EXERC

In [5]:
import pickle 

pickle.dump(model,open('model.pkl' , 'wb'))

In [6]:
model1 = pickle.load(open('model.pkl' , 'rb'))